# Setup

In [1]:
import requests
import json
import pandas as pd
import duckdb

def api_request(endpoint):
    url = f'https://pokeapi.co/api/v2/{endpoint}/'
    response = requests.get(url)
    data = response.json()
    return data

def get_gen_poke(gen):
    data = api_request(f'pokedex/{gen}')
    df = pd.json_normalize(data['pokemon_entries'])
    return df

def get_region_locations(name):
    data = api_request(f'region/{name}')
    df = pd.json_normalize(data['locations'])
    return df

# Main

In [2]:
# 1. Get pokemon from emerald generation
poke_entries = get_gen_poke('hoenn')
poke_entries['id'] = poke_entries['pokemon_species.url'].apply(lambda x: x.split('pokemon-species/')[1].split('/')[0])


In [24]:
# 2. Get emerald region locations
locs = get_region_locations('hoenn')
locs['id'] = locs['url'].apply(lambda x: x.split('/')[-2])

In [ ]:
# Import data into duckdb
try:
    con = duckdb.connect()
    con.execute("CREATE TABLE pokedex AS SELECT * FROM poke_entries")
finally:
    con.close()

# Debug

In [25]:
locs

,name,url,id
0,petalburg-city,https://pokeapi.co/api/v2/location/429/,429
1,slateport-city,https://pokeapi.co/api/v2/location/430/,430
2,lilycove-city,https://pokeapi.co/api/v2/location/431/,431
3,mossdeep-city,https://pokeapi.co/api/v2/location/432/,432
4,sootopolis-city,https://pokeapi.co/api/v2/location/433/,433
...,...,...,...
102,terra-cave,https://pokeapi.co/api/v2/location/803/,803
103,marine-cave,https://pokeapi.co/api/v2/location/804/,804
104,faraway-island,https://pokeapi.co/api/v2/location/812/,812
105,hoenn-battle-frontier,https://pokeapi.co/api/v2/location/813/,813


In [ ]:
poke_entries

In [2]:
# Get location encounter data for each location in the Hoenn region
data = api_request('location-area/429')
# Treat the data
poke_encounters = pd.json_normalize(
    data['pokemon_encounters'],
    record_path = 'version_details',
    meta = [['pokemon', 'name'], ['pokemon', 'url']]
).explode('encounter_details')
encounter_details = poke_encounters['encounter_details'].apply(pd.Series)

In [39]:
import aiohttp
import asyncio

async def get_location_encounters(session, url):
    async with session.get(url) as response:
        return await response.json()

async def get_all_encounters():
    async with aiohttp.ClientSession() as session:
        tasks = [
            get_location_encounters(session, f"https://pokeapi.co/api/v2/location-area/{row.id}/")
            for row in locs.itertuples(index=False)
        ]
        results = await asyncio.gather(*tasks)

        return results

location_encounters = await get_all_encounters()

ContentTypeError: 404, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url='https://pokeapi.co/api/v2/location-area/467/'